In [6]:
from selenium import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import datetime, date, timedelta
import requests, json
import time 


noOfPeople = 2
noOfHours = noOfPeople*2 #quick mafs
roomList = ["Massey Group Study Pod", "309", "Group Study Pod 3", "Group Study Pod 2"]
browserInstances = []

In [12]:
def login(username, password, driver):    
    WebDriverWait(driver, 5).until(EC.visibility_of_element_located(('name','j_username'))).send_keys(username)
    driver.find_element_by_name("j_password").send_keys(password + Keys.RETURN)


def changeDates(driver, date):
    month = date.strftime("%b")
    day = date.strftime("%d")
    year = date.strftime("%Y")
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Go To Date')]")))
    calendarList = driver.find_elements_by_xpath("//button[contains(text(),'Go To Date')]")
    for x in range(len(calendarList)):
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(('xpath',"//button[contains(text(),'Go To Date')]")))
        calendarList = driver.find_elements_by_xpath("//button[contains(text(),'Go To Date')]")
        calendarList[x].click()
        driver.find_element_by_xpath("//div[@class='datepicker-days']").find_element_by_xpath("//th[@class='datepicker-switch']").click()
        WebDriverWait(driver, 1).until(EC.element_to_be_clickable(('xpath',"//span[contains(@class,'month')][text()='" + month + "']"))).click()
        print(day)
        WebDriverWait(driver, 1).until(EC.element_to_be_clickable(('xpath',"//td[contains(@class,'day')][text()='" + day + "']"))).click()
        print("Click Successful")

def checkAvailibility(driver, roomList, dateandtime, totalHours):
    dateToCheck = dateandtime
    driver.get("https://library-calendars.ucl.ac.uk/allspaces")
    changeDates(driver,dateToCheck)
    availability = dict(zip(roomList, [True]*len(roomList)))
    for room in roomList:
        startHour = int(dateToCheck.strftime("%#I"))
        titleString = dateToCheck.strftime(":%Mpm %A, %B %d, %Y - ")
        elementsList = []
        titleString = titleString + room
        for time in range(totalHours): 
            xPath = "//a[@title='" + str(startHour+time) + titleString + "']"
            elementsList.append(driver.find_element_by_xpath(xPath))
        for el in elementsList:
            if(el.get_attribute("class") != "fc-timeline-event fc-h-event fc-event fc-start fc-end s-lc-eq-avail"):
                availability[room] = False
    return availability

In [8]:
def bookRooms(roomList, noOfPeople, startDate, endDate, usernames, passwords):
    driverList = []
    checkingDriver = webdriver.Firefox()
    checkingDriver.implicitly_wait(3);
    chosenRoom = ""
    noOfHours = noOfPeople*2
    #date = "13/03/2019 13"
    startDate = datetime.strptime(startDate, "%d/%m/%Y %H")
    endDate = datetime.strptime(endDate, "%d/%m/%Y")
    dateToCheck = startDate
    for x in range(noOfPeople):
            driver = webdriver.Firefox()
            driverList.append(driver)
            driverList[x].implicitly_wait(5);
            
    for day in range((endDate-startDate).days+2):
        if (dateToCheck.strftime("%w") != "0") and (dateToCheck.strftime("%w") != "6"):
            startHour = int(dateToCheck.strftime("%#I"))
            availability = checkAvailibility(checkingDriver, roomList, dateToCheck, noOfHours)

            for room in roomList: 
                if availability[room] == True: 
                    chosenRoom = room
                    break
                else:
                    continue

            if chosenRoom != "":
                for person in range(noOfPeople):
                    driverList[person].get("https://library-calendars.ucl.ac.uk/allspaces")
                    for hour in range(2):
                        changeDates(driverList[person], dateToCheck)
                        titleString = dateToCheck.strftime(":%Mpm %A, %B %d, %Y - ")
                        titleString = str((person*2)+hour+startHour) + titleString + room
                        print(titleString)
                        xPath = "//a[@title='" + titleString + "']"
                        driverList[person].find_element_by_xpath(xPath).click()
                        WebDriverWait(driverList[person], 5).until(EC.element_to_be_clickable(("id","submit_times"))).click()
                        if day == 0 and hour == 0:
                            login(usernames[person], passwords[person], driverList[person])
                            WebDriverWait(driverList[person], 5).until(EC.element_to_be_clickable(('name','_eventId_proceed'))).click()
                        WebDriverWait(driverList[person], 5).until(EC.element_to_be_clickable(('name','continue'))).click()
                        WebDriverWait(driverList[person], 5).until(EC.element_to_be_clickable(('id','s-lc-eq-bform-submit'))).click()
                        driverList[person].get("https://library-calendars.ucl.ac.uk/allspaces")
            else: 
                print(dateToCheck.strftime("%A %B %d, %Y - ") + room + ": Could not be booked. (No Available Rooms)")
        dateToCheck = dateToCheck + timedelta(days=1)
        chosenRoom = ""
        
    for driver in driverList:
        driver.close()
    checkingDriver.close()

In [13]:
#roomList = ["Massey Group Study Pod", "309", "Group Study Pod 3", "Group Study Pod 2"]
roomList = ["4.04","309"]
usernames = ["", "", "", ""]
passwords = ["", "", "", ""]

# NEVER START ON SATURDAY/SUNDAY AS INPUT
bookRooms(roomList, 4, "28/03/2019 13", "14/04/2019", usernames, passwords)

28
Click Successful
28
Click Successful
28
Click Successful
28
Click Successful
28
Click Successful
28
Click Successful
28
Click Successful


NoSuchElementException: Message: Unable to locate element: //a[@title='1:00pm Thursday, March 28, 2019 - 4.04']
